### Beginning DuckDB  
In this post, I am documenting my learning about DuckDB.   

DuckDB is a relational database that supports the SQL language. Think about your common database solution such as MySQL, etc. However, it lives mostly in memory. No need to connect to a remote destination IP and port. If working with data already written to the disk, then just point duckdb to the **.duckdb** file.

DuckDB is considered a columnar database. While most of us may work with Pandas dataframe directly, DuckDB allows us to run SQL queries against those Pandas DataFrame if needed similarly to what you would have done against a remote SQL database.  

We are able to run queries on data whether that data is on our locally or remotely. 

DuckDB is very useful in environments where there is not much memory  


In [1]:
# First up, let's install DuckDB
!pip install duckdb --upgrade --verbose --no-cache-dir

Using pip 24.2 from /home/securitynik/miniconda3/envs/main_playground/lib/python3.12/site-packages/pip (python 3.12)


In [2]:
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [3]:
# With DuckDB installed, let's import the library
import duckdb

In [4]:
# Let's get the duckdb version
duckdb.__version__

'1.2.1'

In [5]:
# Or alternatively
duckdb.sql(query='CALL pragma_version()')

┌─────────────────┬────────────┐
│ library_version │ source_id  │
│     varchar     │  varchar   │
├─────────────────┼────────────┤
│ v1.2.1          │ 8e52ec4395 │
└─────────────────┴────────────┘

In [6]:
# What platform am I running duckdb on
duckdb.sql(query='CALL pragma_platform()')

┌──────────────────┐
│     platform     │
│     varchar      │
├──────────────────┤
│ linux_amd64_gcc4 │
└──────────────────┘

In [7]:
# Get the user agent
duckdb.sql(query='PRAGMA user_agent')

┌─────────────────────────────────────────────────────┐
│                     user_agent                      │
│                       varchar                       │
├─────────────────────────────────────────────────────┤
│ duckdb/v1.2.1(linux_amd64_gcc4) python/3.12 jupyter │
└─────────────────────────────────────────────────────┘

In [8]:
# Get the current DuckDB settings
duckdb.sql(query='SELECT * FROM duckdb_settings()')

┌─────────────────────────────────────────────┬─────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────┬─────────┐
│                    name                     │      value      │                                                  description                                                   │ input_type │  scope  │
│                   varchar                   │     varchar     │                                                    varchar                                                     │  varchar   │ varchar │
├─────────────────────────────────────────────┼─────────────────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼────────────┼─────────┤
│ access_mode                                 │ automatic       │ Access mode of the database (AUTOMATIC, READ_ONLY or READ_WRITE)                                               │ VARCHAR    │ 

In [9]:
# Set the logging
duckdb.sql(query='SET log_query_path = "/tmp/duckdb.log"')
!ls /tmp/duckdb.log

/tmp/duckdb.log


In [10]:
# Get a list of the available functions
duckdb.sql('PRAGMA functions')

┌────────────┬─────────┬──────────────────────┬─────────┬─────────────┬──────────────┐
│    name    │  type   │      parameters      │ varargs │ return_type │ side_effects │
│  varchar   │ varchar │      varchar[]       │ varchar │   varchar   │   boolean    │
├────────────┼─────────┼──────────────────────┼─────────┼─────────────┼──────────────┤
│ !__postfix │ SCALAR  │ [INTEGER]            │ NULL    │ HUGEINT     │ false        │
│ !~~        │ SCALAR  │ [VARCHAR, VARCHAR]   │ NULL    │ BOOLEAN     │ false        │
│ !~~*       │ SCALAR  │ [VARCHAR, VARCHAR]   │ NULL    │ BOOLEAN     │ false        │
│ %          │ SCALAR  │ [BIGINT, BIGINT]     │ NULL    │ BIGINT      │ false        │
│ %          │ SCALAR  │ [DOUBLE, DOUBLE]     │ NULL    │ DOUBLE      │ false        │
│ %          │ SCALAR  │ [DECIMAL, DECIMAL]   │ NULL    │ DECIMAL     │ false        │
│ %          │ SCALAR  │ [TINYINT, TINYINT]   │ NULL    │ TINYINT     │ false        │
│ %          │ SCALAR  │ [FLOAT, FLOAT]    

In [11]:
# Take it for a test spin
duckdb.sql('SELECT 1').show()

┌───────┐
│   1   │
│ int32 │
├───────┤
│     1 │
└───────┘



With that simple test out of the way, let's get some real data to work with.  

DuckDB is capable of reading data from different formats, such as CSV, Parquet, JSON, Pandas DataFrame, Numpy arrays,  etc. It can read CSV files using the **.read_csv** function or even directly via **.sql**. This **.sql** allows DuckDB to operate on in memory databases.To access an in memory database use the **connect** method without any arguments.


While your CSV, JSON, etc., files are generally stored on your system, there is nothing stopping you from downloading from a remote location via https. If the need arises for you to store your data to disk, as in make it persistent, DuckDB also has facilities for that.  

Let us get some data. For us, let's read the conn.log file from Zeek. This file is serialized in JSON format.

In [12]:
# Read the data and take a quick peak
# Recently, I encountered a problem with loading a very large file where the kernel would crash
# This prevents the kernel from crashing
# https://github.com/duckdb/duckdb/issues/7087
%config SqlMagic.autopandas = False

# Alternatively, ensure ipywidgets is installed in the environment
# !pip install ipywidgets --verbose --no-cache-dir
# https://github.com/duckdb/duckdb/issues/6815

quack = duckdb.read_json(path_or_buffer='/mnt/d/ML/conn-today.json')
quack

┌───────────────────┬────────────────────┬──────────────┬───────────┬────────────────┬───────────┬─────────┬────────────┬────────────┬────────────┬──────────────┬──────────┬───────────┬───────────────┬───────────┬───────────────┬────────────────────────┬────────────┬────────────┬─────────┐
│        ts         │        uid         │  id.orig_h   │ id.orig_p │   id.resp_h    │ id.resp_p │  proto  │ conn_state │ local_orig │ local_resp │ missed_bytes │ history  │ orig_pkts │ orig_ip_bytes │ resp_pkts │ resp_ip_bytes │        duration        │ orig_bytes │ resp_bytes │ service │
│      double       │      varchar       │   varchar    │   int64   │    varchar     │   int64   │ varchar │  varchar   │  boolean   │  boolean   │    int64     │ varchar  │   int64   │     int64     │   int64   │     int64     │         double         │   int64    │   int64    │ varchar │
├───────────────────┼────────────────────┼──────────────┼───────────┼────────────────┼───────────┼─────────┼────────────┼──────

Now that we know we can read data, let's write this data back to the file system for testing. In this case, let's write in the Parquet format.

In [13]:
# Writing to the file system
quack.write_parquet(file_name=r'/tmp/conn-today.pqt')

# With the above done, let's verify the file was successfully written
!ls -l /mnt/d/ML/conn-today.pqt

-rwxrwxrwx 1 securitynik securitynik 1006239 Jan  1 13:54 /mnt/d/ML/conn-today.pqt


We now know we can open files and write to the file system. Let's create persistent database. That is a database that is stored to the file system.  

In [14]:
# Create the database
# Let's actually connect to that database and do some work with it
con = duckdb.connect(database='/tmp/securitynik.duckdb', read_only=False)
con

In [15]:
# Verify if the was created
!ls -l /tmp/securitynik.duckdb

-rw-r--r-- 1 securitynik securitynik 12288 Mar 29 12:51 /tmp/securitynik.duckdb


In [16]:
# Create a table named users
con.sql(query='''CREATE TABLE IF NOT EXISTS users 
        (id INTEGER PRIMARY KEY,
        fname VARCHAR,
        lname VARCHAR,
        username VARCHAR,  
        )
        ''')

In [17]:
# Let's list all tables in the securitynik database
con.sql(query='PRAGMA show_tables')

┌─────────┐
│  name   │
│ varchar │
├─────────┤
│ users   │
└─────────┘

In [18]:
# Query the empty table
con.sql(query='SELECT * FROM users')

┌───────┬─────────────────────┬─────────────────────┬────────────────────────┐
│  id   │        fname        │        lname        │        username        │
│ int32 │       varchar       │       varchar       │        varchar         │
├───────┼─────────────────────┼─────────────────────┼────────────────────────┤
│     1 │ guest_fname         │ guest_lname         │ guest_username         │
│     2 │ anotehr_guest_fname │ anotehr_guest_lname │ anotehr_guest_username │
│     3 │ you                 │ me                  │ somebody               │
│     4 │ guest_fname         │ guest_lname         │ guest_username         │
└───────┴─────────────────────┴─────────────────────┴────────────────────────┘

In [20]:
# Let's insert some records
con.sql('''INSERT INTO users (id, fname, lname, username) VALUES (5, 'guest_fname', 'guest_lname', 'guest_username')''')

In [21]:
# Well I did think that was going to work, as I expect the primary key to auto increment. 
# It did not.
# Let us try this again. This time create a sequence
# Let's overwrite the table we created before, with the new structure

# Create a table named users
con.sql(query='CREATE SEQUENCE IF NOT EXISTS id_sequence START 1')
con.sql(query='''CREATE OR REPLACE TABLE users 
        (id INTEGER PRIMARY KEY DEFAULT nextval('id_sequence'),
        fname VARCHAR,
        lname VARCHAR,
        username VARCHAR,  
        )
        ''')

In [22]:
# Insert the records again
con.sql(query='''INSERT INTO users 
        (fname, lname, username) 
        VALUES ('guest_fname', 'guest_lname', 'guest_username')''')

# Run the questy again to see if this works
con.sql(query='SELECT * FROM users')

┌───────┬─────────────┬─────────────┬────────────────┐
│  id   │    fname    │    lname    │    username    │
│ int32 │   varchar   │   varchar   │    varchar     │
├───────┼─────────────┼─────────────┼────────────────┤
│     4 │ guest_fname │ guest_lname │ guest_username │
└───────┴─────────────┴─────────────┴────────────────┘

In [23]:
# Now we got that to work, let's add two records
con.sql(query='''INSERT INTO users 
        (fname, lname, username) 
        VALUES ('anotehr_guest_fname', 'anotehr_guest_lname', 'anotehr_guest_username'),
        ('you', 'me', 'somebody')'''
        )

# Run the questy again to see if this works
con.sql(query='SELECT * FROM users')

┌───────┬─────────────────────┬─────────────────────┬────────────────────────┐
│  id   │        fname        │        lname        │        username        │
│ int32 │       varchar       │       varchar       │        varchar         │
├───────┼─────────────────────┼─────────────────────┼────────────────────────┤
│     4 │ guest_fname         │ guest_lname         │ guest_username         │
│     5 │ anotehr_guest_fname │ anotehr_guest_lname │ anotehr_guest_username │
│     6 │ you                 │ me                  │ somebody               │
└───────┴─────────────────────┴─────────────────────┴────────────────────────┘

In [25]:
# Well at least we know that works.
# No one wants to enter records one by one
# Exporting some data from Postgres sql
# $ psql --username securitynik --dbname securitynik --command "select * from users;" --csv > /tmp/users.csv
# Read the csv vile
con.sql(query='''INSERT INTO users 
        SELECT * FROM read_csv('/tmp/users.csv', delim=',', header=true, skip=0)
        ON CONFLICT DO NOTHING
        ''')

In [26]:
# Run the query again to see if this works
con.sql(query='SELECT * FROM users ORDER BY id')

┌───────┬─────────────────────┬─────────────────────┬────────────────────────┐
│  id   │        fname        │        lname        │        username        │
│ int32 │       varchar       │       varchar       │        varchar         │
├───────┼─────────────────────┼─────────────────────┼────────────────────────┤
│     1 │ Security            │ Nik                 │ securitynik            │
│     2 │ nik                 │ Alleyne             │ nika                   │
│     3 │ SANS                │ Training            │ sans                   │
│     4 │ guest_fname         │ guest_lname         │ guest_username         │
│     5 │ anotehr_guest_fname │ anotehr_guest_lname │ anotehr_guest_username │
│     6 │ you                 │ me                  │ somebody               │
│     7 │ Saadia              │ K                   │ sk                     │
│     8 │ Tren                │ A                   │ tba                    │
│     9 │ Pam                 │ K                   

In [29]:
# Ok let's create a whole new table 
# $ psql --username securitynik --dbname securitynik --command "select * from bloggers;" --csv  > /tmp/bloggers.csv

con.sql(query='''CREATE OR REPLACE TABLE bloggers
        ( id INTEGER PRIMARY KEY DEFAULT nextval('id_sequence'),  
        username VARCHAR,
        blog_url VARCHAR,
        user_id INTEGER REFERENCES users(id) )
         ''')

# Query the table 
con.sql(query='SELECT * FROM bloggers')

┌───────┬──────────┬──────────┬─────────┐
│  id   │ username │ blog_url │ user_id │
│ int32 │ varchar  │ varchar  │  int32  │
├───────┴──────────┴──────────┴─────────┤
│                0 rows                 │
└───────────────────────────────────────┘

In [30]:
# Load the data from a CSV file to populate the table
con.sql(query=''' INSERT INTO bloggers SELECT * FROM read_csv('/tmp/bloggers.csv', delim=',', header=true, skip=0) ''')

# Query the table 
con.sql(query='SELECT * FROM bloggers')

┌───────┬─────────────┬─────────────────────────┬─────────┐
│  id   │  username   │        blog_url         │ user_id │
│ int32 │   varchar   │         varchar         │  int32  │
├───────┼─────────────┼─────────────────────────┼─────────┤
│     2 │ securitynik │ securitynik.blogspot.ca │       1 │
│     1 │ securitynik │ www.securitynik.com     │       1 │
│     3 │ pk          │ www.pk.local            │       3 │
│     4 │ nta         │ www.nta.local           │       3 │
│     5 │ nsa         │ www.nsa.local           │       4 │
│     6 │ nsa         │ nsa.local               │       4 │
│     7 │ test        │ 404.local               │      10 │
└───────┴─────────────┴─────────────────────────┴─────────┘

In [31]:
# Let's build another table using the information we stored in the database earlier
# If the table exists, then just move on
con.sql('CREATE TABLE IF NOT EXISTS conn_log AS FROM read_parquet("/tmp/conn-today.pqt")')

In [32]:
# Look at the list of databases and their paths
# We created one so the expectation here is only one
con.sql('PRAGMA database_list')

┌───────┬─────────────┬─────────────────────────┐
│  seq  │    name     │          file           │
│ int64 │   varchar   │         varchar         │
├───────┼─────────────┼─────────────────────────┤
│   546 │ securitynik │ /tmp/securitynik.duckdb │
└───────┴─────────────┴─────────────────────────┘

In [33]:
# Get information on the database size
con.sql('CALL pragma_database_size()')

┌───────────────┬───────────────┬────────────┬──────────────┬─────────────┬─────────────┬──────────┬──────────────┬──────────────┐
│ database_name │ database_size │ block_size │ total_blocks │ used_blocks │ free_blocks │ wal_size │ memory_usage │ memory_limit │
│    varchar    │    varchar    │   int64    │    int64     │    int64    │    int64    │ varchar  │   varchar    │   varchar    │
├───────────────┼───────────────┼────────────┼──────────────┼─────────────┼─────────────┼──────────┼──────────────┼──────────────┤
│ securitynik   │ 0 bytes       │     262144 │            0 │           0 │           0 │ 3.3 MiB  │ 13.1 MiB     │ 12.3 GiB     │
└───────────────┴───────────────┴────────────┴──────────────┴─────────────┴─────────────┴──────────┴──────────────┴──────────────┘

In [34]:
# Let's get the memory 
# https://duckdb.org/2024/07/09/memory-management.html
duckdb.sql('FROM duckdb_memory()')

┌──────────────────┬────────────────────┬─────────────────────────┐
│       tag        │ memory_usage_bytes │ temporary_storage_bytes │
│     varchar      │       int64        │          int64          │
├──────────────────┼────────────────────┼─────────────────────────┤
│ BASE_TABLE       │                  0 │                       0 │
│ HASH_TABLE       │                  0 │                       0 │
│ PARQUET_READER   │                  0 │                       0 │
│ CSV_READER       │                  0 │                       0 │
│ ORDER_BY         │                  0 │                       0 │
│ ART_INDEX        │                  0 │                       0 │
│ COLUMN_DATA      │                  0 │                       0 │
│ METADATA         │                  0 │                       0 │
│ OVERFLOW_STRINGS │                  0 │                       0 │
│ IN_MEMORY_TABLE  │                  0 │                       0 │
│ ALLOCATOR        │                  0 │       

In [35]:
# Get the current temporary files
duckdb.sql('FROM duckdb_temporary_files()')

┌─────────┬───────┐
│  path   │ size  │
│ varchar │ int64 │
├─────────┴───────┤
│     0 rows      │
└─────────────────┘

In [36]:
# How about we change the amount for the memory limit of the database
con.sql(query="SET memory_limit = '1GB'")
con.sql('CALL pragma_database_size()')

┌───────────────┬───────────────┬────────────┬──────────────┬─────────────┬─────────────┬──────────┬──────────────┬──────────────┐
│ database_name │ database_size │ block_size │ total_blocks │ used_blocks │ free_blocks │ wal_size │ memory_usage │ memory_limit │
│    varchar    │    varchar    │   int64    │    int64     │    int64    │    int64    │ varchar  │   varchar    │   varchar    │
├───────────────┼───────────────┼────────────┼──────────────┼─────────────┼─────────────┼──────────┼──────────────┼──────────────┤
│ securitynik   │ 0 bytes       │     262144 │            0 │           0 │           0 │ 3.3 MiB  │ 13.1 MiB     │ 953.6 MiB    │
└───────────────┴───────────────┴────────────┴──────────────┴─────────────┴─────────────┴──────────┴──────────────┴──────────────┘

In [37]:
# Let's list all tables in the securitynik database
con.sql('PRAGMA show_tables')

┌──────────┐
│   name   │
│ varchar  │
├──────────┤
│ bloggers │
│ conn_log │
│ users    │
└──────────┘

In [38]:
# Expand that view
# Let's get a little bit more data about the tables in database, tables, columns, etc.
con.sql('PRAGMA show_tables_expanded')

┌─────────────┬─────────┬──────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────┐
│  database   │ schema  │   name   │                                                                                                      column_names                                                                                                      │                                                                               column_types                                                                                │ temporary │
│   varchar   │ varchar │ varchar  │                                                                                        

In [39]:
# Describe the users table
con.sql('DESCRIBE users')

┌─────────────┬─────────────┬─────────┬─────────┬────────────────────────┬─────────┐
│ column_name │ column_type │  null   │   key   │        default         │  extra  │
│   varchar   │   varchar   │ varchar │ varchar │        varchar         │ varchar │
├─────────────┼─────────────┼─────────┼─────────┼────────────────────────┼─────────┤
│ id          │ INTEGER     │ NO      │ PRI     │ nextval('id_sequence') │ NULL    │
│ fname       │ VARCHAR     │ YES     │ NULL    │ NULL                   │ NULL    │
│ lname       │ VARCHAR     │ YES     │ NULL    │ NULL                   │ NULL    │
│ username    │ VARCHAR     │ YES     │ NULL    │ NULL                   │ NULL    │
└─────────────┴─────────────┴─────────┴─────────┴────────────────────────┴─────────┘

In [40]:
# Describe the bloggers table
con.sql('DESCRIBE bloggers')

┌─────────────┬─────────────┬─────────┬─────────┬────────────────────────┬─────────┐
│ column_name │ column_type │  null   │   key   │        default         │  extra  │
│   varchar   │   varchar   │ varchar │ varchar │        varchar         │ varchar │
├─────────────┼─────────────┼─────────┼─────────┼────────────────────────┼─────────┤
│ id          │ INTEGER     │ NO      │ PRI     │ nextval('id_sequence') │ NULL    │
│ username    │ VARCHAR     │ YES     │ NULL    │ NULL                   │ NULL    │
│ blog_url    │ VARCHAR     │ YES     │ NULL    │ NULL                   │ NULL    │
│ user_id     │ INTEGER     │ YES     │ NULL    │ NULL                   │ NULL    │
└─────────────┴─────────────┴─────────┴─────────┴────────────────────────┴─────────┘

In [41]:
# Describe the conn table
con.sql('DESCRIBE conn_log')

┌───────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│  column_name  │ column_type │  null   │   key   │ default │  extra  │
│    varchar    │   varchar   │ varchar │ varchar │ varchar │ varchar │
├───────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ ts            │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ uid           │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ id.orig_h     │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ id.orig_p     │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ id.resp_h     │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ id.resp_p     │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ proto         │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ conn_state    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ local_orig    │ BOOLEAN     │ YES     │ NULL    │ NULL    │ NULL    │
│ local_resp    │ BOOLEAN     │ YES     │ NULL    │ NULL    │ NU

In [42]:
# Get information on the table from a different perspective
con.sql('PRAGMA table_info(conn_log)')

┌───────┬───────────────┬─────────┬─────────┬────────────┬─────────┐
│  cid  │     name      │  type   │ notnull │ dflt_value │   pk    │
│ int32 │    varchar    │ varchar │ boolean │  varchar   │ boolean │
├───────┼───────────────┼─────────┼─────────┼────────────┼─────────┤
│     0 │ ts            │ DOUBLE  │ false   │ NULL       │ false   │
│     1 │ uid           │ VARCHAR │ false   │ NULL       │ false   │
│     2 │ id.orig_h     │ VARCHAR │ false   │ NULL       │ false   │
│     3 │ id.orig_p     │ BIGINT  │ false   │ NULL       │ false   │
│     4 │ id.resp_h     │ VARCHAR │ false   │ NULL       │ false   │
│     5 │ id.resp_p     │ BIGINT  │ false   │ NULL       │ false   │
│     6 │ proto         │ VARCHAR │ false   │ NULL       │ false   │
│     7 │ conn_state    │ VARCHAR │ false   │ NULL       │ false   │
│     8 │ local_orig    │ BOOLEAN │ false   │ NULL       │ false   │
│     9 │ local_resp    │ BOOLEAN │ false   │ NULL       │ false   │
│    10 │ missed_bytes  │ BIGINT  

In [43]:
# Get the storage info for the conn table
# Could do the same for the others
con.sql(query='CALL pragma_storage_info(conn_log)')

┌──────────────┬─────────────┬───────────┬─────────────┬────────────┬──────────────┬───────┬───────┬──────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────┬─────────────┬────────────┬──────────┬──────────────┬──────────────┬──────────────────────┐
│ row_group_id │ column_name │ column_id │ column_path │ segment_id │ segment_type │ start │ count │ compression  │                                                     stats                                                     │ has_updates │ persistent │ block_id │ block_offset │ segment_info │ additional_block_ids │
│    int64     │   varchar   │   int64   │   varchar   │   int64    │   varchar    │ int64 │ int64 │   varchar    │                                                    varchar                                                    │   boolean   │  boolean   │  int64   │    int64     │   varchar    │       int64[]        │
├──────────────┼─────────────┼───────────┼─

In [44]:
# Let's now confirm the data can be accessed from the conn_log table
con.sql('SELECT * FROM conn_log')

┌───────────────────┬────────────────────┬──────────────┬───────────┬────────────────┬───────────┬─────────┬────────────┬────────────┬────────────┬──────────────┬──────────┬───────────┬───────────────┬───────────┬───────────────┬────────────────────────┬────────────┬────────────┬─────────┐
│        ts         │        uid         │  id.orig_h   │ id.orig_p │   id.resp_h    │ id.resp_p │  proto  │ conn_state │ local_orig │ local_resp │ missed_bytes │ history  │ orig_pkts │ orig_ip_bytes │ resp_pkts │ resp_ip_bytes │        duration        │ orig_bytes │ resp_bytes │ service │
│      double       │      varchar       │   varchar    │   int64   │    varchar     │   int64   │ varchar │  varchar   │  boolean   │  boolean   │    int64     │ varchar  │   int64   │     int64     │   int64   │     int64     │         double         │   int64    │   int64    │ varchar │
├───────────────────┼────────────────────┼──────────────┼───────────┼────────────────┼───────────┼─────────┼────────────┼──────

In [45]:
# Try a different command
# Count the number fo records
con.sql('SELECT COUNT(*) FROM conn_log')

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        24048 │
└──────────────┘

In [46]:
# try the execute command to grab one record
con.execute('SELECT * FROM conn_log')
print(con.fetchone())

(1696017594.59247, 'ChefyE3DEW1aClvrpg', '127.0.0.1', 6789, '127.0.0.1', 59106, 'tcp', 'OTH', True, True, 0, 'Ccc', 0, 0, 0, 0, None, None, None, None)


**Getting our SQL Kung-fu on**

In [47]:
# Let's select the first 10 records and order them by the timestamp (ts) column
con.sql('SELECT * FROM conn_log ORDER BY ts LIMIT 10')

┌───────────────────┬────────────────────┬──────────────┬───────────┬────────────────┬───────────┬─────────┬────────────┬────────────┬────────────┬──────────────┬─────────────┬───────────┬───────────────┬───────────┬───────────────┬────────────────────┬────────────┬────────────┬─────────┐
│        ts         │        uid         │  id.orig_h   │ id.orig_p │   id.resp_h    │ id.resp_p │  proto  │ conn_state │ local_orig │ local_resp │ missed_bytes │   history   │ orig_pkts │ orig_ip_bytes │ resp_pkts │ resp_ip_bytes │      duration      │ orig_bytes │ resp_bytes │ service │
│      double       │      varchar       │   varchar    │   int64   │    varchar     │   int64   │ varchar │  varchar   │  boolean   │  boolean   │    int64     │   varchar   │   int64   │     int64     │   int64   │     int64     │       double       │   int64    │   int64    │ varchar │
├───────────────────┼────────────────────┼──────────────┼───────────┼────────────────┼───────────┼─────────┼────────────┼─────────

In [48]:
# How many unique source IPs. are there
con.sql('SELECT DISTINCT COUNT("id.orig_h") FROM conn_log')

┌────────────────────┐
│ count("id.orig_h") │
│       int64        │
├────────────────────┤
│              24048 │
└────────────────────┘

In [49]:
# Get 10 unique source IP addresses
con.sql('SELECT DISTINCT "id.orig_h" FROM conn_log ORDER BY ts LIMIT 10')

┌───────────────────────────┐
│         id.orig_h         │
│          varchar          │
├───────────────────────────┤
│ 192.168.0.62              │
│ 192.168.0.75              │
│ 192.168.0.24              │
│ 192.168.0.31              │
│ fe80::eb3f:74b2:4679:6bae │
│ fe80::8450:9cff:fe16:ef95 │
│ 192.168.0.2               │
│ 192.168.0.78              │
│ 192.168.0.8               │
│ 144.195.33.213            │
├───────────────────────────┤
│          10 rows          │
└───────────────────────────┘

In [50]:
# Get the largest duration value from the conn_log table
con.sql('SELECT MAX(duration) FROM conn_log')

┌────────────────────┐
│   max(duration)    │
│       double       │
├────────────────────┤
│ 115607.98462510109 │
└────────────────────┘

In [51]:
# Get a sum of the duration
# Get the largest duration
con.sql('SELECT SUM(duration) FROM conn_log')

┌───────────────────┐
│   sum(duration)   │
│      double       │
├───────────────────┤
│ 825393.7917554379 │
└───────────────────┘

In [52]:
# Find all records where the duration is greater than 10 minutes
con.sql('SELECT * FROM conn_log WHERE duration > 600')

┌───────────────────┬────────────────────┬──────────────┬───────────┬────────────────┬───────────┬─────────┬────────────┬────────────┬────────────┬──────────────┬──────────────────┬───────────┬───────────────┬───────────┬───────────────┬────────────────────┬────────────┬────────────┬──────────┐
│        ts         │        uid         │  id.orig_h   │ id.orig_p │   id.resp_h    │ id.resp_p │  proto  │ conn_state │ local_orig │ local_resp │ missed_bytes │     history      │ orig_pkts │ orig_ip_bytes │ resp_pkts │ resp_ip_bytes │      duration      │ orig_bytes │ resp_bytes │ service  │
│      double       │      varchar       │   varchar    │   int64   │    varchar     │   int64   │ varchar │  varchar   │  boolean   │  boolean   │    int64     │     varchar      │   int64   │     int64     │   int64   │     int64     │       double       │   int64    │   int64    │ varchar  │
├───────────────────┼────────────────────┼──────────────┼───────────┼────────────────┼───────────┼─────────┼────

In [53]:
# Summarize the table
con.sql(query='SUMMARIZE conn_log')

┌───────────────┬─────────────┬─────────────────────┬───────────────────────────┬───────────────┬────────────────────┬────────────────────┬───────────────────────┬────────────────────────┬─────────────────────┬───────┬─────────────────┐
│  column_name  │ column_type │         min         │            max            │ approx_unique │        avg         │        std         │          q25          │          q50           │         q75         │ count │ null_percentage │
│    varchar    │   varchar   │       varchar       │          varchar          │     int64     │      varchar       │      varchar       │        varchar        │        varchar         │       varchar       │ int64 │  decimal(9,2)   │
├───────────────┼─────────────┼─────────────────────┼───────────────────────────┼───────────────┼────────────────────┼────────────────────┼───────────────────────┼────────────────────────┼─────────────────────┼───────┼─────────────────┤
│ ts            │ DOUBLE      │ 1695902349.570724   

In [54]:
# We might instead want to see the information above as a dataframe
# Let's use the **.df** function to get the data as a dataframe
con.sql('SELECT "id.orig_h", "id.resp_h" FROM conn_log WHERE duration > 600').df()

,id.orig_h,id.resp_h
0,192.168.0.31,165.225.208.44
1,192.168.0.31,165.225.208.44
2,192.168.0.31,165.225.208.44
3,192.168.0.95,34.98.105.85
4,192.168.0.95,34.149.46.130
...,...,...
112,192.168.0.31,192.168.0.4
113,192.168.0.31,192.168.0.4
114,192.168.0.31,192.168.0.4
115,192.168.0.28,52.236.189.131


In [55]:
# Let pull back all the tables and some rows and store these into a dataframe
tmp_df = con.sql('SELECT * FROM conn_log WHERE duration > 600').df()
tmp_df

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,duration,orig_bytes,resp_bytes,service
0,1.696001e+09,CthQmt3D5lmDwrRxUl,192.168.0.31,64113,165.225.208.44,443,tcp,RSTR,True,False,0,ShADadttfr,821,57635,1113,77717,16416.052616,24783,32633,"http,ssl"
1,1.695988e+09,CYUQe22Chebdk2ue1e,192.168.0.31,51760,165.225.208.44,443,tcp,RSTR,True,False,0,ShADdaTtr,17442,8006593,15058,1295176,29656.265580,7308515,692511,"http,ssl"
2,1.695994e+09,CRqfCIG05qUCcFcF,192.168.0.31,55543,165.225.208.44,443,tcp,RSTR,True,False,0,ShADdafr,3150,757605,1603,212389,23924.337346,631593,148257,"http,ssl"
3,1.696017e+09,CfLKAp3pg8ZlCxVyk,192.168.0.95,35630,34.98.105.85,443,tcp,SF,True,False,0,ShADadtFf,20,2284,22,3070,618.818055,1212,1772,ssl
4,1.696017e+09,CSRhux2BwyF9SSzjDf,192.168.0.95,49344,34.149.46.130,443,tcp,SF,True,False,0,ShADadtFf,18,1960,15,6040,610.352108,1004,5213,ssl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,1.695988e+09,CsqLYG90S5u8wUGji,192.168.0.31,55906,192.168.0.4,9200,tcp,OTH,True,True,0,DacAcTcTcT,37698,42754618,9589,383560,29664.822755,41183458,3923632,None
113,1.695988e+09,C3uhyF2oaouFzSjN97,192.168.0.31,56024,192.168.0.4,9200,tcp,OTH,True,True,0,DacAccc,32628,40508275,5091,203640,29650.207311,39203107,4989212,None
114,1.695988e+09,CaXyts2pErtFF9wXa,192.168.0.31,64349,192.168.0.4,8220,tcp,OTH,True,True,0,DcTcTAacTcT,2372,503027,362,14480,29651.999841,406240,25806,None
115,1.696018e+09,CiIl0H14bcFsvU5rl8,192.168.0.28,62619,52.236.189.131,443,tcp,SF,True,False,0,ShADadFRf,36,3259,23,5250,2881.177577,1807,4318,ssl


In [56]:
# Could also get the data as pandas dataframe using fetchdf command
con.sql('SELECT * FROM conn_log WHERE duration > 600').fetchdf()

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,duration,orig_bytes,resp_bytes,service
0,1.696001e+09,CthQmt3D5lmDwrRxUl,192.168.0.31,64113,165.225.208.44,443,tcp,RSTR,True,False,0,ShADadttfr,821,57635,1113,77717,16416.052616,24783,32633,"http,ssl"
1,1.695988e+09,CYUQe22Chebdk2ue1e,192.168.0.31,51760,165.225.208.44,443,tcp,RSTR,True,False,0,ShADdaTtr,17442,8006593,15058,1295176,29656.265580,7308515,692511,"http,ssl"
2,1.695994e+09,CRqfCIG05qUCcFcF,192.168.0.31,55543,165.225.208.44,443,tcp,RSTR,True,False,0,ShADdafr,3150,757605,1603,212389,23924.337346,631593,148257,"http,ssl"
3,1.696017e+09,CfLKAp3pg8ZlCxVyk,192.168.0.95,35630,34.98.105.85,443,tcp,SF,True,False,0,ShADadtFf,20,2284,22,3070,618.818055,1212,1772,ssl
4,1.696017e+09,CSRhux2BwyF9SSzjDf,192.168.0.95,49344,34.149.46.130,443,tcp,SF,True,False,0,ShADadtFf,18,1960,15,6040,610.352108,1004,5213,ssl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,1.695988e+09,CsqLYG90S5u8wUGji,192.168.0.31,55906,192.168.0.4,9200,tcp,OTH,True,True,0,DacAcTcTcT,37698,42754618,9589,383560,29664.822755,41183458,3923632,None
113,1.695988e+09,C3uhyF2oaouFzSjN97,192.168.0.31,56024,192.168.0.4,9200,tcp,OTH,True,True,0,DacAccc,32628,40508275,5091,203640,29650.207311,39203107,4989212,None
114,1.695988e+09,CaXyts2pErtFF9wXa,192.168.0.31,64349,192.168.0.4,8220,tcp,OTH,True,True,0,DcTcTAacTcT,2372,503027,362,14480,29651.999841,406240,25806,None
115,1.696018e+09,CiIl0H14bcFsvU5rl8,192.168.0.28,62619,52.236.189.131,443,tcp,SF,True,False,0,ShADadFRf,36,3259,23,5250,2881.177577,1807,4318,ssl


In [57]:
# We can also convert this data to numpy array if we wish
con.sql('SELECT * FROM conn_log WHERE duration > 600').fetchnumpy()

{'ts': array([1.69600120e+09, 1.69598807e+09, 1.69599381e+09, 1.69601715e+09,
        1.69601719e+09, 1.69601346e+09, 1.69601430e+09, 1.69601431e+09,
        1.69601430e+09, 1.69601430e+09, 1.69601697e+09, 1.69590235e+09,
        1.69600940e+09, 1.69601382e+09, 1.69601381e+09, 1.69601629e+09,
        1.69601629e+09, 1.69601629e+09, 1.69601628e+09, 1.69598578e+09,
        1.69601540e+09, 1.69598647e+09, 1.69601273e+09, 1.69601497e+09,
        1.69601446e+09, 1.69601716e+09, 1.69601749e+09, 1.69601794e+09,
        1.69601685e+09, 1.69601799e+09, 1.69601096e+09, 1.69601641e+09,
        1.69601796e+09, 1.69601811e+09, 1.69601425e+09, 1.69601362e+09,
        1.69601431e+09, 1.69601473e+09, 1.69601412e+09, 1.69601468e+09,
        1.69601346e+09, 1.69601859e+09, 1.69601494e+09, 1.69601503e+09,
        1.69601208e+09, 1.69601869e+09, 1.69601463e+09, 1.69601504e+09,
        1.69601567e+09, 1.69601501e+09, 1.69601635e+09, 1.69601431e+09,
        1.69601846e+09, 1.69601495e+09, 1.69601869e+09, 1.

In [58]:
# Earlier above, we said these dataframes can be queried as if they were SQL tables. Let us verify that
# Finding records that are not TCP traffic
duckdb.sql("SELECT * FROM tmp_df WHERE proto != 'tcp' ")

┌──────────────────┬────────────────────┬─────────────┬───────────┬─────────────┬───────────┬─────────┬────────────┬────────────┬────────────┬──────────────┬─────────┬───────────┬───────────────┬───────────┬───────────────┬───────────────────┬────────────┬────────────┬─────────┐
│        ts        │        uid         │  id.orig_h  │ id.orig_p │  id.resp_h  │ id.resp_p │  proto  │ conn_state │ local_orig │ local_resp │ missed_bytes │ history │ orig_pkts │ orig_ip_bytes │ resp_pkts │ resp_ip_bytes │     duration      │ orig_bytes │ resp_bytes │ service │
│      double      │      varchar       │   varchar   │   int64   │   varchar   │   int64   │ varchar │  varchar   │  boolean   │  boolean   │    int64     │ varchar │   int64   │     int64     │   int64   │     int64     │      double       │   int64    │   int64    │ varchar │
├──────────────────┼────────────────────┼─────────────┼───────────┼─────────────┼───────────┼─────────┼────────────┼────────────┼────────────┼──────────────┼───

In [59]:
# Let's now advance our skills a bit,
# We know from earlier there is a relationship between users and bloggers
# We see the id column is a primary key
con.sql(query='PRAGMA table_info(users)')

┌───────┬──────────┬─────────┬─────────┬────────────────────────┬─────────┐
│  cid  │   name   │  type   │ notnull │       dflt_value       │   pk    │
│ int32 │ varchar  │ varchar │ boolean │        varchar         │ boolean │
├───────┼──────────┼─────────┼─────────┼────────────────────────┼─────────┤
│     0 │ id       │ INTEGER │ true    │ nextval('id_sequence') │ true    │
│     1 │ fname    │ VARCHAR │ false   │ NULL                   │ false   │
│     2 │ lname    │ VARCHAR │ false   │ NULL                   │ false   │
│     3 │ username │ VARCHAR │ false   │ NULL                   │ false   │
└───────┴──────────┴─────────┴─────────┴────────────────────────┴─────────┘

In [60]:
# Similarly, we see the id column is a primary key
con.sql(query='PRAGMA table_info(bloggers)')

┌───────┬──────────┬─────────┬─────────┬────────────────────────┬─────────┐
│  cid  │   name   │  type   │ notnull │       dflt_value       │   pk    │
│ int32 │ varchar  │ varchar │ boolean │        varchar         │ boolean │
├───────┼──────────┼─────────┼─────────┼────────────────────────┼─────────┤
│     0 │ id       │ INTEGER │ true    │ nextval('id_sequence') │ true    │
│     1 │ username │ VARCHAR │ false   │ NULL                   │ false   │
│     2 │ blog_url │ VARCHAR │ false   │ NULL                   │ false   │
│     3 │ user_id  │ INTEGER │ false   │ NULL                   │ false   │
└───────┴──────────┴─────────┴─────────┴────────────────────────┴─────────┘

In [61]:
# We can see the 3rd column below represents show the primary and foreign key columns
con.sql(query='''SELECT * FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE WHERE TABLE_NAME='bloggers' ''')

┌────────────────────┬───────────────────┬──────────────────────────┬───────────────┬──────────────┬────────────┬─────────────┬──────────────────┬───────────────────────────────┐
│ constraint_catalog │ constraint_schema │     constraint_name      │ table_catalog │ table_schema │ table_name │ column_name │ ordinal_position │ position_in_unique_constraint │
│      varchar       │      varchar      │         varchar          │    varchar    │   varchar    │  varchar   │   varchar   │      int64       │             int32             │
├────────────────────┼───────────────────┼──────────────────────────┼───────────────┼──────────────┼────────────┼─────────────┼──────────────────┼───────────────────────────────┤
│ securitynik        │ main              │ bloggers_user_id_id_fkey │ securitynik   │ main         │ bloggers   │ user_id     │                1 │                             1 │
│ securitynik        │ main              │ bloggers_id_pkey         │ securitynik   │ main         │ blog

In [62]:
# Let us create some joins between those two tables now
# but before we do, lets revisit our table outputs so they are infront of us
con.sql(query=''' SELECT * FROM users''')

┌───────┬─────────────────────┬─────────────────────┬────────────────────────┐
│  id   │        fname        │        lname        │        username        │
│ int32 │       varchar       │       varchar       │        varchar         │
├───────┼─────────────────────┼─────────────────────┼────────────────────────┤
│     4 │ guest_fname         │ guest_lname         │ guest_username         │
│     5 │ anotehr_guest_fname │ anotehr_guest_lname │ anotehr_guest_username │
│     6 │ you                 │ me                  │ somebody               │
│     1 │ Security            │ Nik                 │ securitynik            │
│     2 │ nik                 │ Alleyne             │ nika                   │
│     3 │ SANS                │ Training            │ sans                   │
│     7 │ Saadia              │ K                   │ sk                     │
│     8 │ Tren                │ A                   │ tba                    │
│     9 │ Pam                 │ K                   

In [63]:
# Do the same thing with bloggers
con.sql(query=''' SELECT * FROM bloggers''')

┌───────┬─────────────┬─────────────────────────┬─────────┐
│  id   │  username   │        blog_url         │ user_id │
│ int32 │   varchar   │         varchar         │  int32  │
├───────┼─────────────┼─────────────────────────┼─────────┤
│     2 │ securitynik │ securitynik.blogspot.ca │       1 │
│     1 │ securitynik │ www.securitynik.com     │       1 │
│     3 │ pk          │ www.pk.local            │       3 │
│     4 │ nta         │ www.nta.local           │       3 │
│     5 │ nsa         │ www.nsa.local           │       4 │
│     6 │ nsa         │ nsa.local               │       4 │
│     7 │ test        │ 404.local               │      10 │
└───────┴─────────────┴─────────────────────────┴─────────┘

In [64]:
# Perform the inner join, showing the matches on the primary and foreign key
# Only where the two table matches
# https://jordan-hay.github.io/jekyll/update/data/2024/09/22/duckdb.html

con.sql(query=''' SELECT users.id, bloggers.user_id, users.fname, bloggers.blog_url 
        FROM bloggers 
        INNER JOIN users 
        ON users.id=bloggers.user_id
        ORDER BY users.id; ''')

┌───────┬─────────┬─────────────┬─────────────────────────┐
│  id   │ user_id │    fname    │        blog_url         │
│ int32 │  int32  │   varchar   │         varchar         │
├───────┼─────────┼─────────────┼─────────────────────────┤
│     1 │       1 │ Security    │ securitynik.blogspot.ca │
│     1 │       1 │ Security    │ www.securitynik.com     │
│     3 │       3 │ SANS        │ www.pk.local            │
│     3 │       3 │ SANS        │ www.nta.local           │
│     4 │       4 │ guest_fname │ www.nsa.local           │
│     4 │       4 │ guest_fname │ nsa.local               │
│    10 │      10 │ User        │ 404.local               │
└───────┴─────────┴─────────────┴─────────────────────────┘

In [65]:
# Perform the left join
# Return all records on the left and the matching ones on the right
con.sql(query=''' SELECT users.id, bloggers.user_id, users.fname, bloggers.blog_url 
        FROM users
        LEFT JOIN bloggers ON users.id=bloggers.user_id,
        ORDER BY users.id; ''')

┌───────┬─────────┬─────────────────────┬─────────────────────────┐
│  id   │ user_id │        fname        │        blog_url         │
│ int32 │  int32  │       varchar       │         varchar         │
├───────┼─────────┼─────────────────────┼─────────────────────────┤
│     1 │       1 │ Security            │ securitynik.blogspot.ca │
│     1 │       1 │ Security            │ www.securitynik.com     │
│     2 │    NULL │ nik                 │ NULL                    │
│     3 │       3 │ SANS                │ www.nta.local           │
│     3 │       3 │ SANS                │ www.pk.local            │
│     4 │       4 │ guest_fname         │ nsa.local               │
│     4 │       4 │ guest_fname         │ www.nsa.local           │
│     5 │    NULL │ anotehr_guest_fname │ NULL                    │
│     6 │    NULL │ you                 │ NULL                    │
│     7 │    NULL │ Saadia              │ NULL                    │
│     8 │    NULL │ Tren                │ NULL  

In [66]:
# Perform the right join
# Return all records on the right and the matching ones on the left
con.sql(query=''' SELECT users.id, bloggers.user_id, users.fname, bloggers.blog_url 
        FROM users
        RIGHT JOIN bloggers ON users.id=bloggers.user_id,
        ORDER BY users.id; ''')

┌───────┬─────────┬─────────────┬─────────────────────────┐
│  id   │ user_id │    fname    │        blog_url         │
│ int32 │  int32  │   varchar   │         varchar         │
├───────┼─────────┼─────────────┼─────────────────────────┤
│     1 │       1 │ Security    │ securitynik.blogspot.ca │
│     1 │       1 │ Security    │ www.securitynik.com     │
│     3 │       3 │ SANS        │ www.pk.local            │
│     3 │       3 │ SANS        │ www.nta.local           │
│     4 │       4 │ guest_fname │ www.nsa.local           │
│     4 │       4 │ guest_fname │ nsa.local               │
│    10 │      10 │ User        │ 404.local               │
└───────┴─────────┴─────────────┴─────────────────────────┘

In [67]:
# Run the UNION
con.sql(query=''' SELECT users.id, users.fname 
        FROM users WHERE fname='nik' 
        UNION SELECT bloggers.id, blog_url 
        FROM bloggers   ''')

┌───────┬─────────────────────────┐
│  id   │          fname          │
│ int32 │         varchar         │
├───────┼─────────────────────────┤
│     5 │ www.nsa.local           │
│     1 │ www.securitynik.com     │
│     4 │ www.nta.local           │
│     2 │ securitynik.blogspot.ca │
│     3 │ www.pk.local            │
│     2 │ nik                     │
│     6 │ nsa.local               │
│     7 │ 404.local               │
└───────┴─────────────────────────┘

In [68]:
# Run the FULL OUTER
# Return all rows from both tables
con.sql(query=''' SELECT users.id, bloggers.user_id, users.fname 
        FROM users 
        FULL OUTER JOIN bloggers  
        ON users.id=bloggers.user_id ''')

┌───────┬─────────┬─────────────────────┐
│  id   │ user_id │        fname        │
│ int32 │  int32  │       varchar       │
├───────┼─────────┼─────────────────────┤
│     4 │       4 │ guest_fname         │
│     1 │       1 │ Security            │
│     3 │       3 │ SANS                │
│    10 │      10 │ User                │
│     4 │       4 │ guest_fname         │
│     1 │       1 │ Security            │
│     3 │       3 │ SANS                │
│     5 │    NULL │ anotehr_guest_fname │
│     6 │    NULL │ you                 │
│     2 │    NULL │ nik                 │
│     7 │    NULL │ Saadia              │
│     8 │    NULL │ Tren                │
│     9 │    NULL │ Pam                 │
├───────┴─────────┴─────────────────────┤
│ 13 rows                     3 columns │
└───────────────────────────────────────┘

In [69]:
# Another full outer
# This time non-matching records only
con.sql(query=''' SELECT users.id, bloggers.user_id, users.fname 
        FROM users 
        FULL OUTER JOIN bloggers  
        ON users.id=bloggers.user_id 
        WHERE users.id IS NULL OR bloggers.user_id IS NULL''')

┌───────┬─────────┬─────────────────────┐
│  id   │ user_id │        fname        │
│ int32 │  int32  │       varchar       │
├───────┼─────────┼─────────────────────┤
│     5 │    NULL │ anotehr_guest_fname │
│     6 │    NULL │ you                 │
│     2 │    NULL │ nik                 │
│     7 │    NULL │ Saadia              │
│     8 │    NULL │ Tren                │
│     9 │    NULL │ Pam                 │
└───────┴─────────┴─────────────────────┘

In [70]:
# Enough of joins,
# Ending this session
con.close()

In [71]:
# Let us read a file from remote 
# Define the Iris URL
iris_con = duckdb.sql("SELECT * FROM 'https://raw.githubusercontent.com/SecurityNik/Data-Science-and-ML/refs/heads/main/DATASETS/Iris.csv', LIMIT 10")
iris_con

┌───────┬───────────────┬──────────────┬───────────────┬──────────────┬─────────────┐
│  Id   │ SepalLengthCm │ SepalWidthCm │ PetalLengthCm │ PetalWidthCm │   Species   │
│ int64 │    double     │    double    │    double     │    double    │   varchar   │
├───────┼───────────────┼──────────────┼───────────────┼──────────────┼─────────────┤
│     1 │           5.1 │          3.5 │           1.4 │          0.2 │ Iris-setosa │
│     2 │           4.9 │          3.0 │           1.4 │          0.2 │ Iris-setosa │
│     3 │           4.7 │          3.2 │           1.3 │          0.2 │ Iris-setosa │
│     4 │           4.6 │          3.1 │           1.5 │          0.2 │ Iris-setosa │
│     5 │           5.0 │          3.6 │           1.4 │          0.2 │ Iris-setosa │
│     6 │           5.4 │          3.9 │           1.7 │          0.4 │ Iris-setosa │
│     7 │           4.6 │          3.4 │           1.4 │          0.3 │ Iris-setosa │
│     8 │           5.0 │          3.4 │           1.5

In [72]:
# There was no need to download the file first. 
# just extract three columns from the dataframe just to see what can be done
iris_con.to_df()[['SepalLengthCm', 'PetalLengthCm', 'Species']]


,SepalLengthCm,PetalLengthCm,Species
0,5.1,1.4,Iris-setosa
1,4.9,1.4,Iris-setosa
2,4.7,1.3,Iris-setosa
3,4.6,1.5,Iris-setosa
4,5.0,1.4,Iris-setosa
5,5.4,1.7,Iris-setosa
6,4.6,1.4,Iris-setosa
7,5.0,1.5,Iris-setosa
8,4.4,1.4,Iris-setosa
9,4.9,1.5,Iris-setosa


In [73]:
# Make a connection again. This time by setting up the database
iris_con = duckdb.connect() 

# Store the results in a table
iris_con.sql('''SELECT * FROM 'https://raw.githubusercontent.com/SecurityNik/Data-Science-and-ML/refs/heads/main/DATASETS/Iris.csv' ''').to_table('iris_table')

# Present that table
iris_con.table('iris_table').df()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


In [74]:
# Explain a table
print(iris_con.sql(query='EXPLAIN (FORMAT HTML) SELECT SepalLengthCm FROM iris_table'))

┌───────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [75]:
# Get some statistics
# How many rows are there
iris_con.sql(query=''' SELECT COUNT(*) FROM  iris_table''')

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│          150 │
└──────────────┘

In [76]:
# Get the unique species
iris_con.sql(query=''' SELECT DISTINCT(Species) FROM  iris_table''')

┌─────────────────┐
│     Species     │
│     varchar     │
├─────────────────┤
│ Iris-setosa     │
│ Iris-versicolor │
│ Iris-virginica  │
└─────────────────┘

In [77]:
# Get a count
# Get the unique species count
iris_con.sql(query=''' SELECT COUNT ( DISTINCT(Species) ) FROM  iris_table''')

┌─────────────────────────┐
│ count(DISTINCT Species) │
│          int64          │
├─────────────────────────┤
│                       3 │
└─────────────────────────┘

In [78]:
# Get the sum of the SepalLengthCm column
iris_con.sql(query=''' SELECT SUM(SepalLengthCm) FROM  iris_table''')

┌────────────────────┐
│ sum(SepalLengthCm) │
│       double       │
├────────────────────┤
│  876.5000000000002 │
└────────────────────┘

In [79]:
# Multiply all the rows and then sum them up
# Then group them by he Species
iris_con.sql(query=''' SELECT Species, 
             SUM( SepalLengthCm * SepalWidthCm  * PetalLengthCm * PetalWidthCm ) AS specs_prod_sum
             FROM iris_table  
             GROUP BY Species
             
             ''')

┌─────────────────┬────────────────────┐
│     Species     │   specs_prod_sum   │
│     varchar     │       double       │
├─────────────────┼────────────────────┤
│ Iris-setosa     │           320.3581 │
│ Iris-versicolor │  4867.516699999998 │
│ Iris-virginica  │ 11372.547999999997 │
└─────────────────┴────────────────────┘

In [80]:
# Try something different
# Just minus
iris_con.sql(query=''' SELECT Species, 
             SUM( SepalLengthCm - SepalWidthCm  - PetalLengthCm - PetalWidthCm ) AS specs_prod_sum
             FROM iris_table  
             GROUP BY Species
             
             ''')

┌─────────────────┬─────────────────────┐
│     Species     │   specs_prod_sum    │
│     varchar     │       double        │
├─────────────────┼─────────────────────┤
│ Iris-virginica  │ -198.19999999999993 │
│ Iris-setosa     │                -6.0 │
│ Iris-versicolor │              -121.0 │
└─────────────────┴─────────────────────┘

In [81]:
# Get the average of the SepalLengthCm column
iris_con.sql(query=''' SELECT AVG(SepalLengthCm) FROM  iris_table''')

┌────────────────────┐
│ avg(SepalLengthCm) │
│       double       │
├────────────────────┤
│  5.843333333333335 │
└────────────────────┘

In [82]:
# Get the average of SepalLength again but only show two columns and group by those two columns
iris_con.sql(query=''' SELECT PetalLengthCM,Species, 
             AVG(SepalLengthCm) AS avg_sepal 
             FROM  iris_table
             GROUP BY PetalLengthCM,Species 
             ORDER BY avg_sepal''')

┌───────────────┬─────────────────┬────────────────────┐
│ PetalLengthCm │     Species     │     avg_sepal      │
│    double     │     varchar     │       double       │
├───────────────┼─────────────────┼────────────────────┤
│           1.1 │ Iris-setosa     │                4.3 │
│           1.0 │ Iris-setosa     │                4.6 │
│           1.3 │ Iris-setosa     │ 4.8428571428571425 │
│           4.5 │ Iris-virginica  │                4.9 │
│           1.6 │ Iris-setosa     │  4.914285714285714 │
│           1.4 │ Iris-setosa     │  4.916666666666667 │
│           1.9 │ Iris-setosa     │  4.949999999999999 │
│           3.3 │ Iris-versicolor │               4.95 │
│           3.0 │ Iris-versicolor │                5.1 │
│           1.5 │ Iris-setosa     │  5.128571428571428 │
│            ·  │      ·          │                 ·  │
│            ·  │      ·          │                 ·  │
│            ·  │      ·          │                 ·  │
│           6.0 │ Iris-virginic

In [83]:
# Get the MAX of the SepalLengthCm column
iris_con.sql(query=''' SELECT MAX(SepalLengthCm) FROM  iris_table''')

┌────────────────────┐
│ max(SepalLengthCm) │
│       double       │
├────────────────────┤
│                7.9 │
└────────────────────┘

In [84]:
# Get the standard deviation of the SepalLengthCm column
iris_con.sql(query=''' SELECT STDDEV(SepalLengthCm) FROM  iris_table''')

┌───────────────────────┐
│ stddev(SepalLengthCm) │
│        double         │
├───────────────────────┤
│    0.8280661279778637 │
└───────────────────────┘

In [85]:
# Get the standard deviation of the SepalLengthCm column
# Same thing from a different perspective.
# Looking at the sample
iris_con.sql(query=''' SELECT STDDEV_SAMP(SepalLengthCm) FROM  iris_table''')

┌────────────────────────────┐
│ stddev_samp(SepalLengthCm) │
│           double           │
├────────────────────────────┤
│         0.8280661279778637 │
└────────────────────────────┘

In [86]:
# Get the standard deviation of the SepalLengthCm column
# Same thing from a different perspective.
# Looking at the Population
iris_con.sql(query=''' SELECT STDDEV_POP(SepalLengthCm) FROM  iris_table''')

┌───────────────────────────┐
│ stddev_pop(SepalLengthCm) │
│          double           │
├───────────────────────────┤
│        0.8253012917851417 │
└───────────────────────────┘

In [87]:
# Get the variance  of the SepalLengthCm column
iris_con.sql(query=''' SELECT VARIANCE(SepalLengthCm) FROM  iris_table''')

┌─────────────────────────┐
│ variance(SepalLengthCm) │
│         double          │
├─────────────────────────┤
│      0.6856935123042518 │
└─────────────────────────┘

In [88]:
# Get the correlation between the Sepal Length and Sepal Width
iris_con.sql(query=''' SELECT CORR(SepalLengthCm,SepalWidthCm) FROM  iris_table''')

┌───────────────────────────────────┐
│ corr(SepalLengthCm, SepalWidthCm) │
│              double               │
├───────────────────────────────────┤
│              -0.10936924995064935 │
└───────────────────────────────────┘

In [89]:
# Get the correlation between the Petal Length and Petal Width
iris_con.sql(query=''' SELECT CORR(PetalLengthCm,PetalWidthCm) FROM  iris_table''')

┌───────────────────────────────────┐
│ corr(PetalLengthCm, PetalWidthCm) │
│              double               │
├───────────────────────────────────┤
│                0.9627570970509665 │
└───────────────────────────────────┘

In [90]:
# Get the Covariance between the Petal Length and Petal Width
iris_con.sql(query=''' SELECT COVAR_POP(PetalLengthCm,PetalWidthCm) FROM  iris_table''')

┌────────────────────────────────────────┐
│ covar_pop(PetalLengthCm, PetalWidthCm) │
│                 double                 │
├────────────────────────────────────────┤
│                      1.287744888888889 │
└────────────────────────────────────────┘

In [91]:
# Get the Median of PetalLengthCM
iris_con.sql(query=''' SELECT MEDIAN(PetalLengthCm) FROM  iris_table''')

┌───────────────────────┐
│ median(PetalLengthCm) │
│        double         │
├───────────────────────┤
│                  4.35 │
└───────────────────────┘

In [92]:
# Get the Median Absolute Deviation
iris_con.sql(query=''' SELECT MAD(PetalLengthCm) FROM  iris_table''')

┌────────────────────┐
│ mad(PetalLengthCm) │
│       double       │
├────────────────────┤
│               1.25 │
└────────────────────┘

In [93]:
# Get the mode
iris_con.sql(query=''' SELECT MODE(PetalLengthCm) FROM  iris_table''')

┌───────────────────────┐
│ "mode"(PetalLengthCm) │
│        double         │
├───────────────────────┤
│                   1.5 │
└───────────────────────┘

In [94]:
# Get the 50 percentile which is the same as the median
iris_con.sql(query=''' SELECT QUANTILE_CONT(PetalLengthCm, .50) FROM  iris_table''')

┌───────────────────────────────────┐
│ quantile_cont(PetalLengthCm, .50) │
│              double               │
├───────────────────────────────────┤
│                              4.35 │
└───────────────────────────────────┘

In [95]:
# Get the regression intercept and Petal Width
iris_con.sql(query=''' SELECT REGR_INTERCEPT(PetalLengthCm,PetalWidthCm) 
             FROM  iris_table''')

┌─────────────────────────────────────────────┐
│ regr_intercept(PetalLengthCm, PetalWidthCm) │
│                   double                    │
├─────────────────────────────────────────────┤
│                          1.0905721458773812 │
└─────────────────────────────────────────────┘

In [96]:
# Get the regression SLOPE and Petal Width
iris_con.sql(query=''' SELECT REGR_SLOPE(PetalLengthCm,PetalWidthCm) 
             FROM  iris_table''')

┌─────────────────────────────────────────┐
│ regr_slope(PetalLengthCm, PetalWidthCm) │
│                 double                  │
├─────────────────────────────────────────┤
│                        2.22588530655391 │
└─────────────────────────────────────────┘

In [97]:
# Get the values into an array
iris_con.sql(query=''' SELECT ARRAY_AGG(SepalLengthCm) FROM  iris_table''')

┌────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                                                                                                                                                                                                                                                    

In [98]:
# Get the values into an array
# Order by the species
iris_con.sql(query=''' SELECT ARRAY_AGG(SepalLengthCm) FROM  iris_table
             GROUP BY Species ''')

┌────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                                                                                                                  array_agg(SepalLengthCm)                                                                                                                  │
│                                                                                                                          double[]                                                                                                                          │
├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [99]:
# Create a pandas dataframe
iris_df = iris_con.sql(''' SELECT * FROM iris_table,
             LIMIT 10''').df()

iris_df

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
5,6,5.4,3.9,1.7,0.4,Iris-setosa
6,7,4.6,3.4,1.4,0.3,Iris-setosa
7,8,5.0,3.4,1.5,0.2,Iris-setosa
8,9,4.4,2.9,1.4,0.2,Iris-setosa
9,10,4.9,3.1,1.5,0.1,Iris-setosa


In [100]:
# Write the dataframe back to a database table
iris_con.register('securitynik_iris', iris_df)

In [101]:
# Confirm the table is there
iris_con.sql(query=''' PRAGMA show_tables ''')

┌──────────────────┐
│       name       │
│     varchar      │
├──────────────────┤
│ iris_table       │
│ securitynik_iris │
└──────────────────┘

In [102]:
# Well if we opened a connection to a table, we should close it too
iris_con.close()

*References*    
- https://duckdb.org/docs/installation/index?version=stable&environment=python  
- https://duckdb.org/2024/03/29/external-aggregation  
- https://duckdb.org/docs/api/python/data_ingestion.html  
- https://duckdb.org/docs/sql/statements/create_table.html  
- https://duckdb.org/docs/sql/statements/create_table.html  
- https://duckdb.org/why_duckdb
- https://motherduck.com/blog/duckdb-ecosystem-newsletter-november-2024/  
- https://duckdb.org/2024/10/04/duckdb-user-survey-analysis.html  
- https://practicaldataengineering.substack.com/p/building-data-pipeline-using-duckdb  
- https://practicaldataengineering.substack.com/p/duckdb-beyond-the-hype  
- https://duckdb.org/pdf/SIGMOD2019-demo-duckdb.pdf   
- https://medium.com/@bruhhmx/introduction-to-duckdb-7d0b4c0de7f5  
- https://r.duckdb.org/reference/duckdb_register.html  
- https://duckdb.org/docs/configuration/pragmas.html  
- https://duckdb.org/docs/configuration/overview.html   
- https://duckdb.org/docs/guides/overview.html  
- https://duckdb.org/2024/09/25/changing-data-with-confidence-and-acid.html  
- https://maximebeauchemin.medium.com/functional-data-engineering-a-modern-paradigm-for-batch-data-processing-2327ec32c42a  
- https://duckdb.org/docs/api/cli/dot_commands  
- https://duckdb.org/2024/03/29/external-aggregation  
- https://duckdb.org/2022/03/07/aggregate-hashtable.html  
- https://shekhargulati.com/2019/12/15/the-5-minute-introduction-to-duckdb-the-sqlite-for-analytics/
- https://open.spotify.com/episode/7mp1QYRJR5Q8Hg9gHACVGG  